In [ ]:
##### Batas penggunaan model yang sudah jadi #############

In [ ]:
import tensorflow as tf
from tensorflow.keras import models
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
import jsons
from flask import Flask, jsonify, request
from io import BytesIO
from PIL import Image
import io, requests, cv2, numpy as np
from sklearn.metrics import classification_report

In [ ]:
def transform_audio_to_spectogram_to_testing(audio_file):
    #audio to spectogram
    y, sr = librosa.load(audio_file)
    librosa.feature.melspectrogram(y=y, sr=sr)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,
                                               fmax=8000)
    fig, ax = plt.subplots()
    S_dB = librosa.power_to_db(S, ref=np.max)
    img = librosa.display.specshow(S_dB)
            # ax.set(title='Log Mel-frequency spectrogram')
    b = io.BytesIO()
    plt.savefig(b, format='jpg')  #save picture in local memory
    #spectogram to numpy
    b.seek(0)
    file_bytes = np.asarray(bytearray(b.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (512, 512))
    image = image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    hasil = image
    y_pred = np.argmax(model.predict(hasil), axis=1)
    x_accuracy = model.predict(hasil)[0]
    output = max(x_accuracy)
    percentage = str(2*(output - 0.5)*100) + " %"
    if str(y_pred[0]) == "0":
       label_nama = "Depresi"
       accuracy = percentage
    elif str(y_pred[0]) == "1":
       label_nama = "Normal"
       accuracy = percentage
    return label_nama,accuracy


In [ ]:
model_ml = "./predictive_model_v_8.h5"
model = models.load_model(model_ml)

In [ ]:
from flask import Flask
from gevent.pywsgi import WSGIServer

app = Flask(__name__)
@app.route('/predict', methods=['POST'])
def infer_image():
    if request.method == 'POST':
        if 'file' not in request.files:
            return "Masukan hanya jenis file saja"
        file = request.files.get('file')
        try:
            img = transform_audio_to_spectogram_to_testing(file)
            json =  { "status_user":img[0],"status_running":"Sukses","level":img[1]} 
        except:
            json =  { "status_user":"-","status_running":"Gagal"}
        return json
    
@app.route('/', methods=['GET'])
def index():
    return "Selamat datang dimachine learning kami"

if __name__ == '__main__':
    # Debug/Development
    # app.run(debug=True, host="0.0.0.0", port="5000")
    # Production
    http_server = WSGIServer(('127.0.0.1', 5000), app)
    http_server.serve_forever()